In [2]:
import pandas as pd
import numpy as np

In [85]:
#Read Yelp csv file
path='D:/Adarsh_Laptop/studetmater/python/pycon-2016-tutorial/data/yelp.csv'
data = pd.read_csv(path)

In [90]:
data = data[(data.stars == 5) | (data.stars == 1)] 


In [91]:
#Retreive only text and stars columns
X_data = data['text']
y_data = data['stars']

In [92]:
X_data.shape

(4086L,)

In [93]:
y_data.shape

(4086L,)

In [94]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X_data,y_data,random_state=1)
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(3064L,)
(1022L,)
(3064L,)
(1022L,)


In [95]:
#CountVectorizer shall be used to work on text data
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm

<3064x16825 sparse matrix of type '<type 'numpy.int64'>'
	with 237720 stored elements in Compressed Sparse Row format>

In [157]:
#Vectorize Test data
X_test_dtm = vect.transform(X_test)

In [137]:
#MultinomialNB for prediction
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=0.5)
nb.fit(X_train_dtm,y_train)
nb_predict = nb.predict(X_test_dtm)

In [147]:
from sklearn import metrics
print metrics.accuracy_score(y_test,nb_predict)

0.925636007828


In [148]:
print metrics.confusion_matrix(y_test,nb_predict)

[[145  39]
 [ 37 801]]


In [149]:
print metrics.recall_score(y_test,nb_predict)

0.788043478261


In [166]:
len(X_test[nb_predict < y_test])
type(y_test)

pandas.core.series.Series

In [167]:
print metrics.roc_auc_score(y_test,nb_predict)

ValueError: Data is not binary and pos_label is not specified

In [133]:
alpha = np.arange(0,1,0.1)
accuracy_list =[]
for i in alpha:
    nb = MultinomialNB(alpha=i)
    nb.fit(X_train_dtm,y_train)
    nb_predict = nb.predict(X_test_dtm)
    accuracy_list.append(metrics.accuracy_score(y_test,nb_predict))

<5x16825 sparse matrix of type '<type 'numpy.int64'>'
	with 260 stored elements in Compressed Sparse Row format>